In [5]:
import ijson
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)

# Chemin vers le fichier JSON volumineux dans Google Drive
file_path = 'offres_emploi.json'

# Lire le fichier JSON par morceaux
def read_large_json(file, chunksize=10000):
    """
    Cette fonction lit un fichier JSON volumineux par morceaux de taille spécifiée.

    Args:
    file (str): Le chemin du fichier JSON.
    chunksize (int): Le nombre de lignes par morceau.

    Yields:
    list: Une liste d'objets JSON correspondant à un morceau du fichier.
    """
    with open(file, 'r', encoding='utf-8') as f:
        objects = ijson.items(f, 'item')
        chunk = []
        for i, obj in enumerate(objects):
            chunk.append(obj)
            if (i + 1) % chunksize == 0:
                yield chunk
                chunk = []
        if chunk:
            yield chunk

# Fonction de nettoyage des données
def clean_data(df):
    """
    Cette fonction nettoie les données d'un DataFrame en supprimant les lignes avec des valeurs manquantes
    dans certaines colonnes et en transformant certaines colonnes.

    Args:
    df (pd.DataFrame): Le DataFrame à nettoyer.

    Returns:
    pd.DataFrame: Le DataFrame nettoyé.
    """
    # Supprimer les lignes avec des valeurs manquantes dans des colonnes critiques
    initial_count = len(df)
    df.dropna(subset=['intitule', 'dateCreation', 'entreprise_nom'], inplace=True)
    cleaned_count = len(df)

#     # Remplacer les valeurs manquantes des colonnes numériques par 0
#     numeric_columns = df.select_dtypes(include=['float64']).columns
#     df[numeric_columns] = df[numeric_columns].fillna(0)

    print(f"Nettoyage des données: {initial_count} -> {cleaned_count} documents")

    # Fonction pour transformer la colonne dureeTravailLibelle
    def transform_duree_travail_libelle(value):
        if isinstance(value, str):
            if "temps partiel" in value:
                return 24
            elif "temps plein" in value:
                return 35
            else:
                numbers = re.findall(r'\d+', value)
                return int(numbers[0]) if numbers else 0
        else:
            return 0

    # Appliquer la transformation à dureeTravailLibelle
    df['dureeTravailLibelle'] = df['dureeTravailLibelle'].apply(transform_duree_travail_libelle)

    # Calculer la moyenne de dureeTravailLibelle
    duree_moyenne = df['dureeTravailLibelle'].mean()

    # Remplacer les valeurs manquantes de dureeTravailLibelle par la moyenne
    df['dureeTravailLibelle'].fillna(duree_moyenne, inplace=True)

    # Fonction pour extraire le salaire en float
    def extract_salaire(salaire, duree):
        if isinstance(salaire, str) and 'Horaire' in salaire:
            montants_list = re.findall(r'\d+\.\d+', salaire)
            montant = float(montants_list[0]) if montants_list else 0
            heures_travail = duree if duree != 0 else 35
            return round(montant * heures_travail * 4, 2)
        elif isinstance(salaire, str) and 'Mensuel' in salaire:
            montants = [float(m) for m in re.findall(r'\d+\.\d+', salaire)]
            if len(montants) == 2:
                return round(max(montants), 2)
            elif len(montants) == 1:
                return round(montants[0], 2)
        elif isinstance(salaire, str) and 'Annuel' in salaire:
            montants_list = re.findall(r'\d+\.\d+', salaire)
            montant = float(montants_list[0]) if montants_list else 0
            return round(montant / 12, 2)

        return 0

    # Appliquer la fonction d'extraction du salaire
    df['salaire_libelle'] = df.apply(lambda row: extract_salaire(row['salaire_libelle'], row['dureeTravailLibelle']), axis=1)

    # Calculer la moyenne des salaires
    salaire_moyen = df['salaire_libelle'].mean()

    # Remplacer les valeurs manquantes de salaire_libelle par la moyenne
    df['salaire_libelle'].fillna(salaire_moyen, inplace=True)

    # Transformer la colonne lieuTravail_libelle pour garder uniquement la partie après le tiret
    df['lieuTravail_libelle'] = df['lieuTravail_libelle'].apply(lambda x: x.split(' - ')[1].strip() if ' - ' in x else x.strip())

#     # Supprimer les colonnes non souhaitées
#     df.drop(columns=['salaire_complement1', 'salaire_complement2', 'dateActualisation', 'appellationlibelle', 'dureeTravailLibelle', 'id'], inplace=True, errors='ignore')

    return df

# Fonction de conversion des types de données
def convert_data_types(df):
    """
    Cette fonction convertit les types de données dans un DataFrame.

    Args:
    df (pd.DataFrame): Le DataFrame à convertir.

    Returns:
    pd.DataFrame: Le DataFrame avec les types de données convertis.
    """
    # Transformer la colonne dateCreation au format "DD-MM-YYYY"
    df['dateCreation'] = pd.to_datetime(df['dateCreation'], errors='coerce').dt.strftime('%d-%m-%Y')

    return df

# Sauvegarde des données nettoyées dans un fichier CSV
output_file = 'cleaned_offres_emploi.csv'

def save_to_csv(df, output_file, first_chunk):
    """
    Cette fonction sauvegarde un DataFrame dans un fichier CSV.

    Args:
    df (pd.DataFrame): Le DataFrame à sauvegarder.
    output_file (str): Le chemin du fichier CSV.
    first_chunk (bool): Indique s'il s'agit du premier morceau à sauvegarder.
    """
    if first_chunk:
        df.to_csv(output_file, index=False, mode='w', encoding='utf-8', sep=',')
    else:
        df.to_csv(output_file, index=False, mode='a', encoding='utf-8', sep=',', header=False)

# Traitement des données par morceaux
first_chunk = True
total_documents_transformed = 0
total_documents_stored = 0

# Lire et traiter chaque morceau de données
for chunk in read_large_json(file_path):
    # Convertir le morceau en DataFrame
    df_chunk = pd.DataFrame(chunk)

    print(f"Chunk de départ : {len(df_chunk)} documents")
    total_documents_transformed += len(df_chunk)
    try:
        # Nettoyer les données
        df_chunk = clean_data(df_chunk)
        # Convertir les types de données
        df_chunk = convert_data_types(df_chunk)

        # Renommer les colonnes
        df_chunk.rename(columns={
            'lieuTravail_libelle': 'Address',
            'intitule': 'Title',
            'description': 'Description',
            'dateCreation': 'Date',
            'typeContrat': 'Type of contract',
            'salaire_libelle': 'Salary',
            'origineOffre_urlOrigine': 'Link',
            'entreprise_nom': 'Company'
        }, inplace=True)

        # Conserver uniquement les colonnes spécifiées et dans l'ordre souhaité
        columns_to_keep = ['Title', 'Company', 'Description', 'Date', 'Address', 'Salary', 'Link']
        df_chunk = df_chunk[columns_to_keep]

        # Supprimer les doublons
        df_chunk.drop_duplicates(inplace=True)

        total_documents_stored += len(df_chunk)
    except KeyError as e:
        print(f"Erreur KeyError : {e}")
        continue

    # Sauvegarder les données nettoyées dans un fichier CSV
    save_to_csv(df_chunk, output_file, first_chunk)
    first_chunk = False
    print(f"Documents stockés après ce chunk : {total_documents_stored}")

print(f"Nettoyage et sauvegarde des données terminés. Nombre total de documents transformés : {total_documents_transformed}")
print(f"Nombre total de documents réellement stockés : {total_documents_stored}")

# Lire le fichier CSV et afficher le nombre de documents
df_csv = pd.read_csv(output_file)
print(f"Nombre total de documents dans le fichier CSV : {len(df_csv)}")


Chunk de départ : 10000 documents
Nettoyage des données: 10000 -> 9084 documents
Documents stockés après ce chunk : 9084
Chunk de départ : 10000 documents
Nettoyage des données: 10000 -> 9039 documents
Documents stockés après ce chunk : 18123
Chunk de départ : 1233 documents
Nettoyage des données: 1233 -> 1211 documents
Documents stockés après ce chunk : 19334
Nettoyage et sauvegarde des données terminés. Nombre total de documents transformés : 21233
Nombre total de documents réellement stockés : 19334
Nombre total de documents dans le fichier CSV : 19334


In [2]:
!pip3 install ijson

   ---------------------------------------- 0.0/48.2 kB ? eta -:--:--
   ---------------------------------------- 48.2/48.2 kB 2.5 MB/s eta 0:00:00
